In [1]:
import os
os.chdir("..")

In [234]:
from fix_data import add_label_T
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA

In [211]:
df = pd.read_csv("data/DropSeq/HCC1806_Filtered_Normalised_3000_Data_train.txt", sep=" ")
df = add_label_T(df, dropname=True)
df

,H1-5,MALAT1,MT-RNR2,ARVCF,BCYRN1,ATXN7L2,IGFBP3,H1-3,CTIF,RNF123,CCDC97,RPL12P42,TRPT1,PARP12,ABTB2,CRLS1,SYCE1L,REP15,NDST2,TBC1D22A-AS1,GBP2,PRKAA2,H2AC11,RAPGEF3,CARD9,MARCHF3,EGFLAM,ZNF710,H4C3,ANXA6,CTF1,C12orf50,RRAGD,ZSCAN18,DDIT4,HSPA5,MB,TMPRSS13,ZNRF2,H1-2,...,FBXO48,SART1,ZNF865,CTBP2,CAP1,NAA40,RAC1,LIPE-AS1,RPL27,BORCS6,CORO7,SLC38A10,NDUFB10,MRPS36,ASB7,MREG,TSGA10,SETD2,METTL26,SRC,STC2,SLC35C2,TBC1D10A,DDIT3,H3-3A,AP1M1,KEAP1,RAD9A,ZFAS1,BATF3,CDKN3,DLD,PMPCA,ZNF165,SCCPDH,NTAN1,CLIP2,DUSP23,ZNF682,label
0,2,3,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,9,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,5,2,0,0,1,0,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14677,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14678,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14679,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14680,3,3,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,2,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [226]:
df = df.sample(frac=1)

In [227]:
X = df.drop("label", axis=1)
y = df["label"]

In [240]:
def get_batch(n, batch_size):
    start_idx = n * batch_size
    end_idx = (n + 1) * batch_size
    X_batch = X[start_idx:end_idx]
    y_batch = y[start_idx:end_idx]

    return X_batch, y_batch

In [241]:
batch_size = 2447
num_batches = 6

In [255]:
X_train_val, X_test, y_train_val, y_val = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25)

In [260]:
clf = lgb.LGBMClassifier(reg_alpha=0.1)
param_grid = {
    'reg_alpha': [0.0, 0.1, 0.5],
    'num_leaves': [20, 30, 40],
    'max_depth': [5, 10]
}
grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_, grid_search.best_score_)

{'max_depth': 15, 'num_leaves': 40} 0.9451646565998966


In [266]:
clf = LogisticRegression(solver="liblinear")
param_grid = {
    "penalty": ["l1", "l2"],
    "C": [0.1, 1, 10]
}
grid_search = GridSearchCV(clf, param_grid)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_, grid_search.best_score_)

{'C': 0.1, 'penalty': 'l2'} 0.9519767106838094


In [265]:
clf = lgb.LGBMClassifier(max_depth=10, num_leaves=40, reg_alpha=0.1)
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9482465100442629

In [275]:
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9540347293156282

In [272]:
clf = LogisticRegression(penalty="l2", C=0.1, solver="liblinear")
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.9588014981273408